In [54]:
#I installe efficientnet for one of my tests but it did not perform well so it is not needed
#source: https://keras.io/api/applications/efficientnet/

!pip install efficientnet

     |████████████████████████████████| 50 kB 5.0 MB/s eta 0:00:011


### Importing Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import efficientnet.tfkeras as efn
from tensorflow.keras.regularizers import l2


import requests

### Functions section

In [2]:
#This function will send me a message ont elegram when the neural network is done fitting


def telegram_bot_sendtext(bot_message):
    bot_token = '1653403779:AAFxxs8vLirSm-nXejEvmHpBoASuZOO75D8'
    bot_chatID = '1589422433'
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + \
                '&parse_mode=MarkdownV2&text=' + bot_message
    response = requests.get(send_text)
    
    return response.json

### Data Set-up

In [3]:
##setting the directory from which I want to get my data

data_dir = '../data/layer'

In [4]:
#Setting the batch size

batch_size = 256

#The height and width of each of my images in 256 picels so I will hard code that

img_height = 256
img_width = 256

### Train, test, splitting

In [5]:
#Setting training set to 80% of my data
#I'm using the batch_size, img_height, img_width

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 18124 files belonging to 10 classes.
Using 14500 files for training.


In [6]:
#Setting testing set to be 20% of my data

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 18124 files belonging to 10 classes.
Using 3624 files for validation.


In [7]:
class_names = train_ds.class_names
print(class_names)

['bacterial_spot', 'early_blight', 'healthy', 'late_blight', 'leaf_mold', 'leaf_septoria', 'mosaic_virus', 'spider_mites', 'target_spot', 'yellow_leaf_curl_virus']


In [8]:
num_classes = 10

### Layer Set-up For Model

In [9]:
#This is the layer that will augment the data so I have mroe images to work from
#The code flips horizontally, does a random rotation, and random zoom

data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [56]:
# efficient = efn.EfficientNetB0(weights='imagenet', include_top=False, 
#                        input_shape=(256, 256, 3))

# for layer in efficient.layers[:]:
#     layer.trainable = False

16809984/16804768 [==============================] - 1s 0us/step


In [16]:
#Building my model topology

model = Sequential([
  #First layer augments the data
  data_augmentation,
  #efficient,
  #This layer rescales all pixels
  layers.experimental.preprocessing.Rescaling(1./255),
  #The remaining layers are added in random order with dropouts and regularization to reduce overfitting
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu', kernel_regularizer = l2(0.001)),
  layers.MaxPooling2D(),
  layers.Dropout(0.5),  
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, padding = 'same', activation = 'relu'),
  layers.Dropout(0.5),
  layers.Flatten(),
  layers.Dense(num_classes)
])

In [17]:
#Currently this is set to the default learning rate, but I have it here
#so I have the option to change it if I see fit

opt = keras.optimizers.Adam(learning_rate=0.00001)

In [18]:
#Compiling model with the adam optimizer and learning rate I set int eh previous cell
#Using accuracy as the metric

model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [19]:
#Quick model summary to see what it will look like visually

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
rescaling_1 (Rescaling)      (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 64, 32)       

### Model Fitting

In [ ]:
epochs = 50
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

telegram_bot_sendtext('done')

Epoch 1/50
57/57 [==============================] - 148s 3s/step - loss: 2.2138 - accuracy: 0.2483 - val_loss: 2.1592 - val_accuracy: 0.2953
Epoch 2/50
57/57 [==============================] - 147s 3s/step - loss: 2.1079 - accuracy: 0.2962 - val_loss: 2.1217 - val_accuracy: 0.2953
Epoch 3/50
57/57 [==============================] - 147s 3s/step - loss: 2.0662 - accuracy: 0.2980 - val_loss: 2.0545 - val_accuracy: 0.2969
Epoch 4/50
57/57 [==============================] - 145s 3s/step - loss: 1.9820 - accuracy: 0.3097 - val_loss: 1.9461 - val_accuracy: 0.3562
Epoch 5/50
57/57 [==============================] - 147s 3s/step - loss: 1.8423 - accuracy: 0.3953 - val_loss: 1.8242 - val_accuracy: 0.5251
Epoch 6/50
57/57 [==============================] - 146s 3s/step - loss: 1.7275 - accuracy: 0.4717 - val_loss: 1.7177 - val_accuracy: 0.5615
Epoch 7/50
57/57 [==============================] - 147s 3s/step - loss: 1.6343 - accuracy: 0.5125 - val_loss: 1.6395 - val_accuracy: 0.5712
Epoch 8/50
57

In [15]:
#This cell assigns the accuracy, val accuracy, loss, and val loss to variables
#for plotting
#Matplotlib is used to plot the loss and accuracy curves on the y axis and epochs on the x axis

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

NameError: name 'history' is not defined